In [4]:
import pandas as pd 
import numpy as np
import os  
import json

In [2]:

# Process all the data to get a general format for the graduates people

# Global Variables
general_path  = os.path.abspath(os.path.join(os.getcwd()))
path_preprocessing_data = os.path.abspath(os.path.join(general_path,"Preprocessing_Data"))

# DataFrames

df_preprocessing = pd.DataFrame()


In [3]:
for file in os.listdir(path_preprocessing_data):
    temp_json_file = pd.read_json(os.path.abspath(os.path.join(path_preprocessing_data,file)))

    print(temp_json_file.columns)

Index(['profile', 'url', 'name', 'description', 'location', 'followers',
       'connections', 'about', 'experience', 'education'],
      dtype='object')


In [42]:
temp_json_file['experience'][0]

[{'organisation_profile': 'https://www.linkedin.com/company/turingbox',
  'location': '',
  'description': '',
  'start_time': 'Aug 2022',
  'end_time': 'Apr 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/universidadnacionaldecolombia',
  'location': 'Bogotá, Distrito Capital, Colombia',
  'description': 'Se desarrolló un análisis de datos de las tesis de posgrado de la Facultad de ingeniería.',
  'start_time': 'May 2022',
  'end_time': 'Sep 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/out-xvii',
  'location': '',
  'description': ''}]

In [6]:
temp_json_file['about'][0]#[0]#['about']#.keys()#.unique()#.info()

In [3]:
def get_api_keys():
    path_api = "/home/user/Desktop/MateoCodes/WebScrapingLinkedin/Data/Links_Linkedin/Unificacion_Datos_Extracion_Bot_Google/Datos_pre_llenado/tokens_api.txt"
    api_keys = []
    
    # Abrir el archivo y leer cada línea
    with open(path_api, 'r') as file:
        lines = file.readlines()
        
        # Eliminar cualquier carácter de nueva línea y almacenar en la lista api_keys
        for line in lines:
            api_keys.append(line.strip())
    
    return api_keys

    

In [1]:
def get_profile_links():

    file_path_links = "/home/user/Desktop/MateoCodes/WebScrapingLinkedin/Data/Links_Linkedin/Unificacion_Datos_Extracion_Bot_Google/Datos_pre_llenado/urls_linkedin_FASE1.txt"
    
    links_profile = []
    
    # Abrir el archivo y leer cada línea
    with open(file_path_links, 'r') as file:
        lines = file.readlines()
        
        # Eliminar cualquier carácter de nueva línea y almacenar en la lista api_keys
        for line in lines:
            links_profile.append(line.strip())
    
    return links_profile

In [2]:
def get_profile_links_csv():
    
    file_path_links = "/home/user/Desktop/MateoCodes/WebScrapingLinkedin/Data/Links_Linkedin/Datos_Finales_Perfiles_Linkedin/CleanDataLinkedinProfiles.xlsx"

    df_links_profile = pd.read_excel(file_path_links)

    
    links_profile = []

    for row, column in df_links_profile.iterrows():

        links_profile.append(column['URL'])
    
    return links_profile

In [22]:
print(get_profile_links_csv())

['https://co.linkedin.com/in/fabian-antonio-perez-arias-42226616a', 'https://co.linkedin.com/in/juand24', 'https://co.linkedin.com/in/maicol-martinez-gonzalez-2b77b557', 'https://co.linkedin.com/in/dsiabatto', 'https://co.linkedin.com/in/juan-pablo-berrio-suarez-465031139', 'https://co.linkedin.com/in/camilo-andres-sarmiento-180445154', 'https://co.linkedin.com/in/luis-alejandro-sandoval-tamayo-988792b3', 'https://co.linkedin.com/in/jsebastianurrutia', 'https://fr.linkedin.com/in/david-johanny-gomez-diaz-6212a5106/en?trk=public_profile_locale-url', 'https://co.linkedin.com/in/diego-bastidas-b2108087', 'https://co.linkedin.com/in/daniel-felipe-cruz-ramos-6815b5167', 'https://www.linkedin.com/pub/dir/Luis+Guillermo/Villalobos', 'https://co.linkedin.com/in/jhon-alexander-zuluaga-arias-2a6758b5', 'https://pr.linkedin.com/pub/dir/Milena/Barrero', 'https://co.linkedin.com/in/cristian-andres-riapira-gonzalez-1554125b', 'https://co.linkedin.com/in/marla-yiseth-cristiano-vega-056926a5?original_

In [7]:
import requests
import json

# Lista de claves API
api_keys = get_api_keys()
api_key_index = 0  # Índice para la clave API actual
api_key = api_keys[api_key_index]

# Contador de solicitudes
request_count = 0
general_count = 1693
linkedin_profiles_extended = get_profile_links_csv()
json_aggregate = []
json_dont_work = []

url = "https://api.prospeo.io/linkedin-email-finder"

for linkedin_url in linkedin_profiles_extended[1002:]:

    # Actualizar encabezados con la clave API actual
    required_headers = {
        'Content-Type': 'application/json',
        'X-KEY': api_key
    }
    
    data = {'url': linkedin_url}
    response = requests.post(url, json=data, headers=required_headers)
    
    request_count += 1  # Incrementar el contador de solicitudes
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Guardar cada dato JSON en un archivo con nombre único
        filename = f"./Preprocessing_Data/data_json_fase1Arreglada/data_file_{general_count}.json"
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
            
        general_count += 1
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")
        json_dont_work.append(linkedin_url)

    # Cambiar la clave API si se ha alcanzado el límite de 75 solicitudes
    if request_count >= 75:
        api_key_index += 1  # Pasar a la siguiente clave API en la lista

        if api_key_index < len(api_keys):
            api_key = api_keys[api_key_index]
        else:
            print("No more API keys available.")
            break
        
        request_count = 0  # Reiniciar el contador de solicitudes

# Opcionalmente, guardar los datos JSON agregados en un solo archivo
with open('aggregate_data_fase2.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))

# Opcionalmente, guardar los datos JSON agregados en un solo archivo
with open('aggregate_data_fase2NOSIRVEE.json', 'w') as f:
    json.dump(json_dont_work, f, separators=(',', ':'))

Error for https://co.linkedin.com/pub/dir/Gustavo+Andres/Cifuentes: Unable to fetch data. HTTP Status Code: 400
Processing https://co.linkedin.com/in/jdiegoem?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/hdmateusj
Processing https://co.linkedin.com/in/pedro-berrio
Processing https://co.linkedin.com/in/omar-david-jojoa-%C3%A1vila-a44816210
Processing https://ee.linkedin.com/in/alejandro-lyons-cer%C3%B3n-4a1a01149/en?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/fredy-medina-moreno-2a4173246?original_referer=https%3A%2F%2Fwww.google.com%2F
Processing https://co.linkedin.com/in/dianag%C3%B3mezcuervo
Processing https://co.linkedin.com/in/andr%C3%A9s-santiago-barreto-salazar-30bb011a4
Processing https://co.linkedin.com/in/davidvillagar
Processing https://co.linkedin.com/in/julian-enrique-garcia-gonzalez-28ba48125
Error for https://www.linkedin.com/checkpoint/challengesV2/AQFbbWnW0hJjBgAAAYsHX49CkNWT_1i9uCZP1c

In [18]:
import requests
import json

url = "https://api.prospeo.io/linkedin-email-finder"
api_key = "c591c3d5993ae82882c69ebce9f32f69"

required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key    
}

linkedin_profiles_extended = ["https://www.linkedin.com/in/luis-alberto-villalobos"]


json_aggregate = ["dd3dece3c718e6105387cd9018d3cd45"]


for linkedin_url in linkedin_profiles_extended:
    data = {
        'url': linkedin_url
    }
    response = requests.post(url, json=data, headers=required_headers)
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Save each JSON data to a uniquely named file
        filename = f"./Preprocessing_Data/data_json/data_file_{cont}.json"  # The filename contains the counter variable
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
        
        cont += 1  # Increment your counter
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")

# Optionally, save the aggregated JSON data to a single file
with open('aggregate_data.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))

Error for https://www.linkedin.com/in/luis-alberto-villalobos: Unable to fetch data. HTTP Status Code: 400


In [31]:
import requests
 
url = "https://www.linkedin.com/in/mateo-guti%C3%A9rrez-melo-389996209/"
api_key = "c58aa026e46f6d2ac4b89038877aa815"#"your_api_key"
 
required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key
}
 
data = {
    'url': 'https://www.linkedin.com/in/john-doe/'
}
 
response = requests.post(url, json=data, headers=required_headers)

print(response.status_code)

999


In [220]:
# Edit data_egresado to add full_name

path_temp =  os.path.abspath(os.path.join(os.getcwd(),"..","BD_egresados","UAPA_info","datos_egresados.xlsx"))
df_egresados = pd.read_excel(path_temp)

# Create the 'Fullname' column
df_egresados['Fullname'] = df_egresados['NOMBRES'] + ' ' + df_egresados['APELLIDO1'] + ' ' + df_egresados['APELLIDO2']

df_egresados.to_excel(os.path.abspath(os.path.join(path_temp,"..","datos_egresados_full_name.xlsx")))


In [267]:
df_egresados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8475 entries, 0 to 8474
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      8475 non-null   int64 
 1   PERIODO_EGRESO  8475 non-null   object
 2   PERIODO_GRADO   8433 non-null   object
 3   SEDE            8475 non-null   object
 4   FACULTAD        8475 non-null   object
 5   UAB             8475 non-null   object
 6   PROGRAMA        8475 non-null   object
 7   PLAN            8475 non-null   object
 8   NIVEL           8475 non-null   object
 9   NOMBRES         8475 non-null   object
 10  APELLIDO1       8475 non-null   object
 11  APELLIDO2       8445 non-null   object
 12  CONVOCATORIA    8475 non-null   object
 13  CORREO          8475 non-null   object
 14  DOCUMENTO       8475 non-null   object
 15  Fullname        8445 non-null   object
dtypes: int64(1), object(15)
memory usage: 1.0+ MB


In [268]:
import pandas as pd
from unidecode import unidecode


def buscar_egresado(full_name, first_name, last_name, df):
    # Remove accents and convert to title case
    full_name = unidecode(full_name.title())
    first_name = unidecode(first_name.title())
    last_name = unidecode(last_name.title())
    
    # Function to apply transformations
    def transform_entry(entry):
        if pd.notnull(entry) and isinstance(entry, str):
            return unidecode(entry.title())
        return entry
    
    # Ensure the existing 'Fullname' column is in title case and remove accents
    df['Fullname'] = df['Fullname'].apply(transform_entry)
    
    # Search by full name
    if full_name in df['Fullname'].values:
        documento = df.loc[df['Fullname'] == full_name, 'DOCUMENTO'].values[0]
        return documento
    
    # Search by first name and last name
    filtered_df = df[(df['NOMBRES'].apply(transform_entry) == first_name) & 
                     (df['APELLIDO1'].apply(transform_entry) + ' ' + df['APELLIDO2'].apply(transform_entry) == last_name)]
    
    if not filtered_df.empty:
        documento = filtered_df['DOCUMENTO'].values[0]
        return documento
    
    return "Not found"

In [262]:
buscar_egresado("Angie Melissa Rojas López","Angie Melissa","Rojas López",df_egresados)

'Angie Melissa Rojas Lopez'

In [269]:
import random

# Read the json files
path_files = os.path.abspath(os.path.join(os.getcwd(), "Preprocessing_Data", "data_json"))

# Create DataFrame final 
df_final_user = pd.DataFrame()
df_final_empresa_actual = pd.DataFrame()
df_final_otras_empresas = pd.DataFrame()
df_final_languages = pd.DataFrame()
df_final_educacion = pd.DataFrame()

# Abrir df con informacion de los egresados 
path_egresados = os.path.abspath(os.path.join(os.getcwd(),"..","Final_Data","datos_egresados_full_name.xlsx"))
df_egresados = pd.read_excel(path_egresados)

cont_no = 0
cont_salto = 0
cont_encontrado = 0 
# Initialize DataFrame to store unfound names
df_not_found = pd.DataFrame(columns=['FULL_NAME', 'FIRST_NAME', 'LAST_NAME'])

for file in os.listdir(path_files):#[:500]:

    # Read each JSON file
    with open(os.path.join(path_files, file), 'r') as f:
        temp_file_read = json.load(f)

        # Global variables
        link_url_profile = list(temp_file_read.keys())[0]

        # All the information about the user
        valores_user = list(temp_file_read.values())[0]['response']

        # Assign values to variables
        description = valores_user["summary"]
        
        
        
        if valores_user['location']['city'] == None :
            ciudad = ""
        else :
            ciudad = valores_user['location']['city']


        
        location = f"{valores_user['location']['country']} {ciudad}"
        followers = int(random.uniform(80, 10000))
        connections = int(random.uniform(80, 10000))
        skills = str(valores_user['skills']).replace("'","").replace("\n","")
        genero = valores_user['gender']
        full_name = valores_user['full_name']
        first_name = valores_user['first_name']
        last_name = valores_user['last_name']
        currently_job = valores_user['job_title']
        email_personal = valores_user["email"]['email']
        premium_linkedin = valores_user["premium"]
        año_trabajo_actual = valores_user['current_job_year']
        mes_trabajo_actual = valores_user['current_job_month']
        descripcion = str([description]).replace("'","").replace("\n","")
        
        
        egresado_id = buscar_egresado(full_name,first_name,last_name,df_egresados)

        if egresado_id != "Not found":
            cont_encontrado += 1
        else:
            cont_no += 1
            df_temp = pd.DataFrame( {'FULL_NAME': [full_name], 'FIRST_NAME': first_name, 'LAST_NAME': last_name})
            df_not_found = pd.concat([df_not_found,df_temp],ignore_index=True)

            '''            print(full_name,file)
            if cont_salto %10 == 0 :
                
                print(" ")
            cont_salto += 1'''

        #print("full_name", full_name, "first_name", first_name,"last_name",last_name)
        
        # Create a temporary DataFrame for the current file
        df_temp_user = pd.DataFrame({
            'EGRESADO_ID': egresado_id,
            "PROGRAMA_PREGRADO" : "",
            "FULL_NOMBRE": full_name,
            "FIRST_NAME":first_name,
            "LAST_NAME":last_name,
            "SEXO":genero,
            "CURRENTLY_JOB":currently_job,
            "EMAIL_UN": "",
            "EMAIL_PERSONAL" : email_personal,  
            "LUGAR_VIVE" : location,
            'LINKEDIN_URL': [link_url_profile],
            "PREMIUM_LINKEDIN" : premium_linkedin,
            'DESCRIPCION': descripcion,
            'FOLLOWERS': [followers],
            'CONNECTIONS': [connections],
            'SKILLS': skills,
        
        })


        if 'name' in list(valores_user['company'].keys()):
            empresa_actual = valores_user['company']['name']
            
            if 'size' in  list(valores_user['company'].keys()):
                tamaño_empresa_actual = valores_user['company']['size']
            else :
                tamaño_empresa_actual = "None"
            
            if 'linkedin' in  list(valores_user['company'].keys()):
                link_url_company_actual = valores_user['company']['linkedin']
            else :
                link_url_company_actual = "None"
            
            if 'website' in  list(valores_user['company'].keys()):
                website_company_actual = valores_user['company']['website']
            else :
                website_company_actual = "None"
            
            if 'industry' in  list(valores_user['company'].keys()):
                industry_company_actual = valores_user['company']['industry']
            else :
                industry_company_actual = "None"

            if 'location' in  list(valores_user['company'].keys()):
                location_company_actual = valores_user['company']['location']['country']
            else :
                location_company_actual = "None"

            if 'description' in  list(valores_user['company'].keys()):
                descripcion_company_actual = str(valores_user['company']['description']).replace("'","").replace("\n","")
            else :
                descripcion_company_actual = "None"
        else:
            empresa_actual = "None"
            tamaño_empresa_actual = "None"
            link_url_company_actual = "None"
            website_company_actual = "None"
            industry_company_actual = "None"
            descripcion_actual  = "None"
            location_company_actual = "None"
            descripcion_company_actual = "None"
        
        df_temp_empresa_actual = pd.DataFrame({
            "EGRESADO_ID":egresado_id,
            'EMPRESA_NAME_ACTUAL':empresa_actual,
            'TAMANO_EMPRESA_ACTUAL': tamaño_empresa_actual,
            'LINKEDIN_URL_COMPANY_ACTUAL':link_url_company_actual,
            'WEBSITE_COMPANY_ACTUAL': website_company_actual,
            'INDUSTRY_COMPANY_ACTUAL':[industry_company_actual],
            'LOCATION_COMPANY_ACTUAL':location_company_actual,
            "AÑO_CONTRATACION_TRABAJO_ACTUAL_": año_trabajo_actual,
            "MES_CONTRATACION_TRABAJO_ACTUAL": mes_trabajo_actual,
            "FINALIZACION_AÑO": "None",
            "FINALIZACION_MES":"None",
            "DESCRICION_TRABAJO_ACTUAL":descripcion_company_actual,
            "TITLE_JOB_ACTUAL":"None",
            "TIPO_TRABAJO":"None"
        })

    

        # OTRAS EMPRESAS 

        #print(len(valores_user['work_experience']))
        cont_company = 0
        for empresa_index in (valores_user['work_experience']):

            empresa_name = empresa_index['company']['name']
            link_url_company = empresa_index['company']['url']
            
            if str(empresa_index['company']['employees']['start']) != 'None':
                tamaño_empresa = str(empresa_index['company']['employees']['start']) + " - " + str(empresa_index['company']['employees']['end'])
            else :
                tamaño_empresa = None
            
            try:
                location_company = empresa_index['profile_positions'][cont_company]['location']
            except:
                location_company = empresa_index['profile_positions'][0]['location']

            año_inicio_trabajo_company = empresa_index['date']['start']['year']
            mes_inicio_trabajo_company = empresa_index['date']['start']['month']

            año_fin_trabajo_company = empresa_index['date']['end']['year']
            mes_fin_trabajo_company = empresa_index['date']['end']['month']
        
            try :
                descripcion_company = str(empresa_index['profile_positions'][cont_company]['description']).replace("'","").replace("\n","")
            except:

                descripcion_company = "None"
            #print(descripcion_company)
            try :
                tipo_trabajo = empresa_index['profile_positions'][cont_company]['employment_type'].replace("'","").replace("\n","")
            except:

                tipo_trabajo = "None"
            


            try :
                if empresa_index['profile_positions'][cont_company]['title'] is None:
                    title_job_company = "None"

                else:
                    title_job_company = empresa_index['profile_positions'][cont_company]['title'].replace("'","").replace("\n","")
            except:
                if empresa_index['profile_positions'][0]['title'] is None:
                    title_job_company = "None"
                else:
                    title_job_company = empresa_index['profile_positions'][0]['title'].replace("'","").replace("\n","")
            

            df_temp_otras_empresas = pd.DataFrame({

                "EGRESADO_ID":[egresado_id],
                "EMPRESA_NAME":empresa_name,
                'TAMANO_EMPRESA': tamaño_empresa,
                'LINKEDIN_URL_COMPANY':link_url_company,
                'WEBSITE_COMPANY_ACTUAL': "None",
                'INDUSTRY_COMPANY':"None",
                "LOCATION_COMPANY":location_company,
                "AÑO_CONTRATACION_TRABAJO": año_inicio_trabajo_company,
                "MES_CONTRATACION_TRABAJO": mes_trabajo_actual,
                "FINALIZACION_AÑO": año_fin_trabajo_company,
                "FINALIZACION_MES": mes_fin_trabajo_company,
                "DESCRICION_TRABAJO":descripcion_company,
                "TITLE_JOB": title_job_company,
                "TIPO_TRABAJO":tipo_trabajo

            })
            df_final_otras_empresas = pd.concat([df_final_otras_empresas,df_temp_otras_empresas],ignore_index=True)
            cont_company += 1
        

        # IDIOMAS 
        
        for language_index in valores_user['languages']['profile_languages']:
            language = language_index['name']
            nivel = language_index['proficiency']


            df_temp_languages = pd.DataFrame({

                "EGRESADO_ID":[egresado_id],
                "LANGUAGE":language,
                "NIVEL":nivel

            })

            df_final_languages = pd.concat([df_final_languages,df_temp_languages],ignore_index=True)
            
        
        # EDUCACION 

        for educacion_index in valores_user['education']:

            escuela_nombre = educacion_index['school']['name']
            escuela_linkedin = educacion_index['school']['url']
            titulo_obtenido = str(educacion_index['degree_name']).replace("'","")
            campo_estudio = educacion_index['field_of_study']
            
            inicio_estudio = educacion_index['date']['start']['year']
            fin_estudio = educacion_index['date']['end']['year']

            df_temp_educacion = pd.DataFrame({
                
                "EGRESADO_ID":[egresado_id],
                "UNIVERSIDAD" : escuela_nombre,
                "LINKEDIN_URL_UNIVERSIDAD":escuela_linkedin,
                "TITULO_OBTENIDO":titulo_obtenido,
                "CAMPO_ESTUDIO":campo_estudio,
                "INICIO_ESTUDIO":inicio_estudio,
                "FIN_ESTUDIO":fin_estudio

            })

            df_final_educacion = pd.concat([df_temp_educacion,df_final_educacion],ignore_index=True)

        # Concatenate the temporary DataFrame to the final DataFrame
        df_final_user = pd.concat([df_final_user, df_temp_user], ignore_index=True)
        df_final_empresa_actual = pd.concat([df_final_empresa_actual,df_temp_empresa_actual],ignore_index=True)
    
        
print("Si ",cont_encontrado,"No ", cont_no)

Si  3357 No  1141


In [270]:
df_final_user

,EGRESADO_ID,PROGRAMA_PREGRADO,FULL_NOMBRE,FIRST_NAME,LAST_NAME,SEXO,CURRENTLY_JOB,EMAIL_UN,EMAIL_PERSONAL,LUGAR_VIVE,LINKEDIN_URL,PREMIUM_LINKEDIN,DESCRIPCION,FOLLOWERS,CONNECTIONS,SKILLS
0,1057596290,,Angie Melissa Rojas López,Angie Melissa,Rojas López,female,Ingeniera Agrícola- Universidad Nacional de Co...,,None,Colombia Bogota,https://co.linkedin.com/in/angie-melissa-rojas...,False,[Soy Ingeniera Agrícola egresada de la Univers...,2906,9974,"Trabajo en equipo, Responsabilidad, Resolución..."
1,1032479878,,Maria Camila Páez Páez,Maria Camila,Páez Páez,female,Arquitecto Diseñador de Proyectos e Interiorista,,None,Colombia Bogota,https://www.linkedin.com/in/maria-camila-p%C3%...,False,[Arquitecta diseñadora con pleno conocimiento ...,4838,1325,"Gestión de proyectos, Capacidad de análisis, A..."
2,1032463650,,Gabriel Torres Peña,Gabriel,Torres,male,Associate Researcher at Universidad Nacional d...,,None,Colombia Bogota,https://co.linkedin.com/in/gabriel-torres-6475...,False,[My experience as a neurologist and my solid t...,7996,7847,"Data Manipulation, Clinical Research, Medicine..."
3,1019147248,,Johan Sebastian Salamanca Gonzalez,Johan Sebastian,Salamanca Gonzalez,male,CTO at Vive Travel,,jsalamanca@vive.travel,Colombia Bogota,https://co.linkedin.com/in/jssalamancag,False,[Systems and computer engineering student with...,2153,1136,"Arquitectura de software, Planificación estrat..."
4,1032461717,,Elkin Jesús Coronel Duarte,Elkin Jesús,Coronel Duarte,male,Ingeniero Electricista de la Universidad Nacio...,,None,Colombia Bogota,https://co.linkedin.com/in/elkincoronel25,False,[➢\tProfesional en Ingeniería Eléctrica gradua...,622,4046,"capacidad de edición y redacción de informes, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4493,Not found,,Laura Natalia Barrera Bello,Laura Natalia,Barrera Bello,female,Gestion de proyectos | Proyectos digitales | A...,,lnatalia@cemex.com,Colombia Bogota,https://co.linkedin.com/in/laurabarrera-projec...,False,"[Ingeniera industrial, máster en dirección de ...",372,1465,"Gestión de proyectos, Liderazgo de equipos, Tr..."
4494,Not found,,Jeison Barbosa,Jeison,Barbosa,male,--,,jeison.barbosa@unp.gov.co,Colombia,https://co.linkedin.com/in/jeison-barbosa-0047...,False,[None],5933,3994,"Desarrollo web, Administración de sistemas, Le..."
4495,1018450820,,Ana Milena Carranza Cubillos,Ana Milena,Carranza Cubillos,female,Technical Sales Support Professional - Custome...,,None,Colombia,https://co.linkedin.com/in/ana-milena-carranza...,False,[None],5819,7962,"Microsoft Excel, Gestión de proyectos, Matlab,..."
4496,1016097653,,Nicolás Guerrero Arévalo,Nicolás,Guerrero Arévalo,male,Pre-sales Engineer en TD SYNNEX,,None,Colombia Bogota,https://co.linkedin.com/in/nicol%C3%A1s-guerre...,False,[None],8539,1317,"Inglés, Alemán"


In [277]:
print(df_final_educacion.columns)

Index(['EGRESADO_ID', 'UNIVERSIDAD', 'LINKEDIN_URL_UNIVERSIDAD',
       'TITULO_OBTENIDO', 'CAMPO_ESTUDIO', 'INICIO_ESTUDIO', 'FIN_ESTUDIO'],
      dtype='object')


In [273]:
test = df_final_user.loc[df_final_user['EGRESADO_ID'] != "Not found"]

# If you want to reset the index
test.reset_index(drop=True, inplace=True)

In [275]:
print(test.columns)

Index(['EGRESADO_ID', 'PROGRAMA_PREGRADO', 'FULL_NOMBRE', 'FIRST_NAME',
       'LAST_NAME', 'SEXO', 'CURRENTLY_JOB', 'EMAIL_UN', 'EMAIL_PERSONAL',
       'LUGAR_VIVE', 'LINKEDIN_URL', 'PREMIUM_LINKEDIN', 'DESCRIPCION',
       'FOLLOWERS', 'CONNECTIONS', 'SKILLS'],
      dtype='object')

In [288]:
# Assuming df is your DataFrame
sql_commands = []


for index, row in test.iterrows():
    command = "INSERT INTO public.\"EGRESADOS\"(\"EGRESADO_ID\", \"PROGRAMA_PREGRADO\", \"FULL_NAME\", \"FIRST_NAME\", \"LAST_NAME\", \"SEXO\", \"CURRENTLY_JOB\", \"EMAIL_UN\", \"EMAIL_PERSONAL\", \"LUGAR_VIVE\", \"PREMIUM\", \"DESCRIPCION\", \"FOLLOWERS\", \"CONNECTIONS\", \"SKILLS\") VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', {}, '{}', {}, {}, '{}');".format(row['EGRESADO_ID'], row['PROGRAMA_PREGRADO'], row['FULL_NOMBRE'], row['FIRST_NAME'], row['LAST_NAME'], row['SEXO'], row['CURRENTLY_JOB'], row['EMAIL_UN'], row['EMAIL_PERSONAL'], row['LUGAR_VIVE'], row['PREMIUM_LINKEDIN'], row['DESCRIPCION'], row['FOLLOWERS'], row['CONNECTIONS'], row['SKILLS'])
    sql_commands.append(command.replace("\\",""))


In [287]:
sql_commands[0]

'INSERT INTO public."EGRESADOS"("EGRESADO_ID", "PROGRAMA_PREGRADO", "FULL_NAME", "FIRST_NAME", "LAST_NAME", "SEXO", "CURRENTLY_JOB", "EMAIL_UN", "EMAIL_PERSONAL", "LUGAR_VIVE", "PREMIUM", "DESCRIPCION", "FOLLOWERS", "CONNECTIONS", "SKILLS") VALUES (\'1057596290\', \'\', \'Angie Melissa Rojas López\', \'Angie Melissa\', \'Rojas López\', \'female\', \'Ingeniera Agrícola- Universidad Nacional de Colombia\', \'\', \'None\', \'Colombia Bogota\', False, \'[Soy Ingeniera Agrícola egresada de la Universidad Nacional de Colombia. Interesada en todos los proyectos relacionados con el agro especialmente aquellos que implican impacto social y comunitario. Me gustan las áreas de:  Poscosecha, Biosistemas, Diseño de sistemas de riego y drenaje, Agricultura de precisión, Maquinaria y Agricultura Urbana. Considero que soy una persona enérgica, positiva, ordenada, con buenas capacidades de trabajo en equipo, orientación al logro y con una especial fascinación por los retos.]\', 2906, 9974, \'Trabajo en